<div class="alert alert-block alert-info">
<i>Hello User!<br></i>
Welcome to <b>python-iLOrest-library</b> <br> 
This script provides an example implementation that allows you to <b>retrieve the network interface card (NIC) configuration</b> of your HPE iLO server.<br> 
To perform this you need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library">python-ilorest-library</a> installed in your machine.<br>
And please keep your username and password handy!
<br>
<b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class = "alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">1.</div>
    <br>
    First, we are importing certain files that provide us the functionality to get the nic configuration of the server.</div>

In [ ]:
import sys
import json
from redfish import LegacyRestClient
from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">2.</div><br>
    Below is the get_ilo_nic function. The get_ilo_nic function is responsible for retrieving the network interface card (NIC) configuration of an HPE iLO server. It takes a REST object and a parameter indicating whether to retrieve active or inactive NICs as input. Here's a brief overview of the function's steps:<br>
<ol>The function first obtains the resource directory of the iLO server using the provided REST object. This directory contains information about the server's available resources, including the NICs.</ol>
<ol>It then searches for the NIC-related information within the resource directory by iterating through the instances and identifying the ones related to the server's manager.
Using the obtained NIC information, the function retrieves the NIC configuration data from the server.</ol>
<ol>Depending on the value of the get_active parameter, the function filters the retrieved NICs to either active or inactive ones.</ol>
<ol>Finally, the function outputs the relevant NIC information, including the NIC's self URL and its JSON representation.</ol>
<p>In summary, the get_ilo_nic function allows you to fetch and categorize the NIC configuration of an HPE iLO server based on its active or inactive status. This information can be useful for monitoring and managing your server's network settings.</p>
</div>

In [ ]:
def get_ilo_nic(restobj, get_active):
    resource_instances = get_resource_directory(restobj)
    if resource_instances:
        #Get URI from resource directory
        for instance in resource_instances:
            if "Manager." in instance.Type:
                manager_path = instance.href
                break

    tmp = restobj.get(manager_path)
    response = restobj.get(tmp.dict["links"]["EthernetNICs"]["href"])

    for nic in response.dict["Items"]:
        if get_active and nic["Status"]["State"] == "Enabled":
            sys.stdout.write("Active\t" + nic["links"]["self"]["href"] + \
                                            ": " + json.dumps(nic) + "\n")
        elif get_active == False and nic["Status"]["State"] == "Disabled":
            sys.stdout.write("InActive\t" + nic["links"]["self"]["href"] + \
                                            ": " + json.dumps(nic) + "\n")

<div class = "alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">3.</div><br>
    
This script establishes a connection to an HPE iLO server using the provided URL, login account, and password. It then logs in to the server and retrieves the NIC configuration. The script specifically fetches active NICs and displays their information. Finally, it logs out from the server.
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    #Create a REST object
    REST_OBJ = LegacyRestClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
    REST_OBJ.login()
    sys.stdout.write("\nEXAMPLE 20: Get iLO's NIC configuration\n")
    get_ilo_nic(REST_OBJ, True)
    REST_OBJ.logout()

<div class = "alert alert-block alert-warning">
    <b>When running on the server locally</b> use the following values<br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>